In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/reid/deep-person-reid
!python setup.py develop
import torch
from torch import nn
import torchreid
%cd ..

/content/drive/MyDrive/reid/deep-person-reid
running develop
running egg_info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
reading manifest file 'torchreid.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'torchreid.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.7/torchreid/metrics/rank_cylib/rank_cy.cpython-37m-x86_64-linux-gnu.so -> torchreid/metrics/rank_cylib
Creating /usr/local/lib/python3.7/dist-packages/torchreid.egg-link (link to .)
Adding torchreid 1.4.0 to easy-install.pth file

Installed /content/drive/MyDrive/reid/deep-person-reid
Processing dependencies for torchreid==1.4.0
Searching for isort==4.3.21
Reading https://pypi.org/simple/isort/
Best match: isort 4.3.21
Processing isort-4.3.21-py2.py3-none-any.whl
Installing isort-4.3.21-py2.py3-none-any.wh

In [3]:
datamanager = torchreid.data.datamanager.ImageDataManager(
    root='',
    sources='dukemtmcreid',
    height=256,
    width=128,
    transforms='random_flip',
    batch_size_train=64,
    batch_size_test=64
)
# return train loader of source data
train_loader = datamanager.train_loader

# return test loader of target data
test_loader = datamanager.test_loader

# return train loader of target data
train_loader_t = datamanager.train_loader_t


Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------


  **************** Summary ****************
  source            : ['dukemtmcreid']
  # source datasets : 1
  # source ids      : 702
  # source images   : 16522
  # source cameras  : 8
  target            : ['dukemtmcreid']
  *****************************************




In [ ]:
for target in list(test_loader.keys()):

  print(test_loader[target]['gallery'])
print(test_loader.values)

<built-in method values of dict object at 0x7f738c9e5190>


Model osnet_x1_0

In [ ]:
model = torchreid.models.osnet_x1_0(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model = model.cuda()

weight_path = 'pretrained/osnet_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model, weight_path)
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0015
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"


In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x1_0_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 0.911 (1.075)	data 0.001 (0.143)	eta 0:46:03	loss 6.1969 (6.3562)	acc 9.3750 (9.6875)	lr 0.001500
epoch: [1/10][20/258]	time 0.908 (0.991)	data 0.001 (0.072)	eta 0:42:17	loss 6.1192 (6.2449)	acc 12.5000 (10.3125)	lr 0.001500
epoch: [1/10][30/258]	time 0.911 (0.964)	data 0.001 (0.048)	eta 0:40:57	loss 5.6831 (6.1175)	acc 10.9375 (10.7292)	lr 0.001500
epoch: [1/10][40/258]	time 0.908 (0.951)	data 0.001 (0.036)	eta 0:40:15	loss 5.0742 (5.9098)	acc 23.4375 (12.7734)	lr 0.001500
epoch: [1/10][50/258]	time 0.916 (0.944)	data 0.001 (0.029)	eta 0:39:48	loss 4.9128 (5.6846)	acc 15.6250 (15.3750)	lr 0.001500
epoch: [1/10][60/258]	time 0.915 (0.939)	data 0.000 (0.024)	eta 0:39:27	loss 4.4704 (5.4864)	acc 28.1250 (17.1875)	lr 0.001500
epoch: [1/10][70/258]	time 0.918 (0.936)	data 0.001 (0.021)	eta 0:39:10	loss 4.3592 (5.3007)	acc 29.6875 (19.5312)	lr 0.001500
epoch: [1/10][80/258]	time 0.916 (0.934)	data 0.001 (0.019)	eta 0:38:55	loss 4.5072 (5.1387)	acc 31.2500 (22.0117

Model osnet_x0_75

In [5]:
model_1 = torchreid.models.osnet_x0_75(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model_1 = model_1.cuda()

weight_path_1 = 'pretrained/osnet_x0_75_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model_1, weight_path_1)
optimizer_1 = torchreid.optim.build_optimizer(
    model_1, optim='adam', lr=0.0015
)
scheduler_1 = torchreid.optim.build_lr_scheduler(
    optimizer_1,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x0_75_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"


In [7]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model_1, optimizer_1, scheduler=scheduler_1
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x0_75_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 0.918 (3.723)	data 0.206 (3.000)	eta 2:39:27	loss 2.0776 (2.1074)	acc 79.6875 (80.0000)	lr 0.001500
epoch: [1/10][20/258]	time 0.690 (3.116)	data 0.001 (2.397)	eta 2:12:56	loss 1.8617 (2.0257)	acc 79.6875 (81.0156)	lr 0.001500
epoch: [1/10][30/258]	time 0.716 (2.925)	data 0.001 (2.210)	eta 2:04:18	loss 2.0932 (1.9879)	acc 78.1250 (82.0312)	lr 0.001500
epoch: [1/10][40/258]	time 0.693 (2.752)	data 0.001 (2.036)	eta 1:56:28	loss 1.7047 (1.9705)	acc 85.9375 (82.1094)	lr 0.001500
epoch: [1/10][50/258]	time 7.434 (2.816)	data 6.719 (2.101)	eta 1:58:43	loss 1.9747 (1.9620)	acc 82.8125 (82.4688)	lr 0.001500
epoch: [1/10][60/258]	time 0.710 (2.794)	data 0.001 (2.080)	eta 1:57:21	loss 1.6610 (1.9535)	acc 93.7500 (82.6302)	lr 0.001500
epoch: [1/10][70/258]	time 5.698 (2.757)	data 4.994 (2.043)	eta 1:55:19	loss 1.6024 (1.9294)	acc 90.6250 (83.3482)	lr 0.001500
epoch: [1/10][80/258]	time 0.707 (2.662)	data 0.001 (1.948)	eta 1:50:56	loss 1.8140 (1.9162)	acc 85.9375 (83.78

Model osnet_x0_5

In [4]:
model_2 = torchreid.models.osnet_x0_5(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model_2 = model_2.cuda()

weight_path_2 = 'pretrained/osnet_x0_5_duke_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model_2, weight_path_2)
optimizer_2 = torchreid.optim.build_optimizer(
    model_2, optim='adam', lr=0.0015
)
scheduler_2 = torchreid.optim.build_lr_scheduler(
    optimizer_2,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x0_5_duke_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth"


In [5]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model_2, optimizer_2, scheduler=scheduler_2
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x0_5_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 0.549 (7.059)	data 0.000 (6.464)	eta 5:02:22	loss 6.1270 (6.3245)	acc 9.3750 (9.2188)	lr 0.001500
epoch: [1/10][20/258]	time 1.265 (5.836)	data 0.714 (5.259)	eta 4:09:01	loss 6.0277 (6.1724)	acc 7.8125 (9.2969)	lr 0.001500
epoch: [1/10][30/258]	time 1.753 (6.012)	data 1.200 (5.437)	eta 4:15:30	loss 5.4934 (6.0253)	acc 18.7500 (11.2500)	lr 0.001500
epoch: [1/10][40/258]	time 0.667 (5.629)	data 0.125 (5.059)	eta 3:58:18	loss 5.3343 (5.8312)	acc 17.1875 (13.2812)	lr 0.001500
epoch: [1/10][50/258]	time 0.548 (5.790)	data 0.001 (5.222)	eta 4:04:09	loss 4.4471 (5.6142)	acc 31.2500 (15.7812)	lr 0.001500
epoch: [1/10][60/258]	time 0.546 (5.582)	data 0.000 (5.014)	eta 3:54:25	loss 4.3753 (5.4210)	acc 32.8125 (18.0208)	lr 0.001500
epoch: [1/10][70/258]	time 0.549 (5.703)	data 0.000 (5.136)	eta 3:58:34	loss 3.6302 (5.2206)	acc 39.0625 (20.7589)	lr 0.001500
epoch: [1/10][80/258]	time 0.551 (5.554)	data 0.000 (4.988)	eta 3:51:24	loss 3.7123 (5.0371)	acc 45.3125 (23.7305)	

Model osnet_x0_25

In [6]:
model_3 = torchreid.models.osnet_x0_25(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model_3 = model_3.cuda()

weight_path_3 = 'pretrained/osnet_x0_25_duke_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model_3, weight_path_3)
optimizer_3 = torchreid.optim.build_optimizer(
    model_3, optim='adam', lr=0.0015
)
scheduler_3 = torchreid.optim.build_lr_scheduler(
    optimizer_3,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x0_25_duke_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth"


In [7]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model_3, optimizer_3, scheduler=scheduler_3
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x0_25_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 0.341 (0.519)	data 0.002 (0.137)	eta 0:22:13	loss 6.5459 (6.4834)	acc 9.3750 (10.6250)	lr 0.001500
epoch: [1/10][20/258]	time 0.355 (0.434)	data 0.001 (0.069)	eta 0:18:31	loss 6.4510 (6.4127)	acc 0.0000 (9.5312)	lr 0.001500
epoch: [1/10][30/258]	time 0.335 (0.401)	data 0.001 (0.047)	eta 0:17:03	loss 5.5748 (6.2523)	acc 14.0625 (9.8438)	lr 0.001500
epoch: [1/10][40/258]	time 0.330 (0.386)	data 0.000 (0.035)	eta 0:16:19	loss 5.1557 (6.0163)	acc 20.3125 (12.0312)	lr 0.001500
epoch: [1/10][50/258]	time 0.322 (0.376)	data 0.001 (0.029)	eta 0:15:51	loss 4.5683 (5.7850)	acc 26.5625 (14.4375)	lr 0.001500
epoch: [1/10][60/258]	time 0.326 (0.370)	data 0.001 (0.024)	eta 0:15:33	loss 3.9571 (5.5349)	acc 42.1875 (17.6302)	lr 0.001500
epoch: [1/10][70/258]	time 0.332 (0.365)	data 0.001 (0.021)	eta 0:15:17	loss 4.3569 (5.3201)	acc 29.6875 (20.4464)	lr 0.001500
epoch: [1/10][80/258]	time 0.327 (0.362)	data 0.001 (0.018)	eta 0:15:05	loss 3.7487 (5.1328)	acc 42.1875 (23.2422)	

Evaluation


In [ ]:

model_1 = torchreid.models.osnet_x1_0(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model_1 = model_1.cuda()

weight_path = 'log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10'
torchreid.utils.torchtools.load_pretrained_weights(model_1, weight_path)
optimizer_1 = torchreid.optim.build_optimizer(
    model_1, optim='adam', lr=0.0015
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer_1,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10"


In [ ]:
engine_1 = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model_1, optimizer_1, scheduler=scheduler
)
engine_1.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=1,
    test_only=True,
    visrank=True,
    visrank_topk=10,
    save_dir='log/osnet_x1_0_dukemtmcreid_top10',
    print_freq=10
)

##### Evaluating dukemtmcreid (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 2228-by-512 matrix
Extracting features from gallery set ...
Done, obtained 17661-by-512 matrix
Speed: 0.0963 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 63.9%
CMC curve
Rank-1  : 83.4%
Rank-5  : 91.7%
Rank-10 : 93.2%
Rank-20 : 95.0%
# query: 2228
# gallery 17661
Visualizing top-10 ranks ...
- done 100/2228
- done 200/2228
- done 300/2228
- done 400/2228
- done 500/2228
- done 600/2228
- done 700/2228
- done 800/2228
- done 900/2228
- done 1000/2228
- done 1100/2228
- done 1200/2228
- done 1300/2228
- done 1400/2228
- done 1500/2228
- done 1600/2228
- done 1700/2228
- done 1800/2228
- done 1900/2228
- done 2000/2228
- done 2100/2228
- done 2200/2228
Done. Images have been saved to "log/osnet_x1_0_dukemtmcreid_top10/visrank_dukemtmcreid" ...


In [ ]:
!python deep-person-reid/tools/visualize_actmap.py --root ./ -d dukemtmcreid -m osnet_x1_0 --weights log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10 --save-dir log/visactmap_osnet_x1_0_dukemtmcreid_test

Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader 